# Comet Opik

Opik is an open source end-to-end [LLM Evaluation Platform](https://www.comet.com/site/products/opik/?utm_source=langchain&utm_medium=docs&utm_content=provider_intro_paragraph&utm_campaign=opik) that helps developers track their LLM prompts and responses during both development and production. Users can define and run evaluations to test their LLMs apps before deployment to check for hallucinations, accuracy, context retrevial, and more!

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width=200 />

Check out the colab to see an end to end example on how you can leverage [Opik](https://www.comet.com/site/products/opik/?utm_source=langchain&utm_medium=docs&utm_content=second_paragraph&utm_campaign=opik) to track and evaluate your LLM Applications you are building with Langchain!

<a target="_blank" href="https://colab.research.google.com/github/comet-ml/opik/blob/main/apps/opik-documentation/documentation/docs/cookbook/langchain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



## Install and Configure Opik

In [ ]:
%pip install opik langchain langchain_openai




: 

Configuring Opik to work on your local server address or log data to our hosted version with your [Opik API Key](https://www.comet.com/signup?from=llm&utm_source=langchain&utm_medium=docs&utm_content=api_key&utm_campaign=opik)

In [ ]:
%opik configure 

## Using OpikTracer 

Here is a basic example of how to use the `OpikTracer` callback with a LangChain chain

In [ ]:
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from opik.integrations.langchain import OpikTracer

# Initialize the tracer
opik_tracer = OpikTracer()

# Create the LLM Chain using LangChain
llm = OpenAI(temperature=0)

prompt_template = PromptTemplate(
    input_variables=["input"],
    template="Translate the following text to French: {input}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Generate the translations
translation = llm_chain.run("Hello, how are you?", callbacks=[opik_tracer])
print(translation)

The  `OpikTracer` will automatically log the input prompt, the output, and any metadata for each step in the chain


![opik_langchain](docs/static/img/opik_langchain_example.png)